In [108]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import skew, kurtosis
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

## Read in data

In [109]:
merged_df_original = pd.read_sas("merged_df.sas7bdat", encoding='ISO-8859-1')
signals_original = pd.read_sas("signals_raw_plus.sas7bdat", encoding='ISO-8859-1')

In [110]:
merged_df = merged_df_original.copy()
signals = signals_original.copy()

In [111]:
permnos = merged_df['permno'].unique()
permno_to_gvkey = merged_df.set_index('permno')['gvkey'].to_dict()

In [112]:
# convert yyyymm to datetime
merged_df['date'] = pd.to_datetime(merged_df['yyyymm'].astype(int).astype(str), format='%Y%m')
merged_df['mktcap'] = merged_df['PRC'] * merged_df['SHROUT'] / 1000

In [113]:
len(merged_df['permno'].unique())

1497

In [114]:
january_filter = (merged_df['date'].dt.month == 1)
january_data = merged_df[january_filter]

# Group by permno and check if any January data point has mkt_val or mktcap < 100
valid_permnos = january_data.groupby('permno').apply(
    lambda group: ((group['PRC'] > 5) & (group['mktcap'] >= 100)).all()
)
valid_permnos = valid_permnos[valid_permnos].index

# Filter the dataframe to include only the valid permnos
merged_df = merged_df[merged_df['permno'].isin(valid_permnos)]

merged_df.head()

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/3874959622.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valid_permnos = january_data.groupby('permno').apply(


,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,date,mktcap
1224,10104.0,198602.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.364103,0.064516,-0.136364,0.263158,0.194444,-0.034884,0.301205,0.425926,1986-02-01,NaN
1225,10104.0,198603.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064516,-0.136364,0.263158,0.194444,-0.034884,0.301205,0.425926,0.142857,1986-03-01,275.320375
1226,10104.0,198604.0,76.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,0.636488,...,-0.136364,0.263158,0.194444,-0.034884,0.301205,0.425926,0.142857,0.068182,1986-04-01,329.725000
1227,10104.0,198605.0,77.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,NaN,...,0.263158,0.194444,-0.034884,0.301205,0.425926,0.142857,0.068182,0.159574,1986-05-01,309.941500
1228,10104.0,198606.0,78.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,NaN,...,0.194444,-0.034884,0.301205,0.425926,0.142857,0.068182,0.159574,-0.183486,1986-06-01,321.481875


In [115]:
signals.rename(columns={'PERMNO':'permno'}, inplace=True)
signals['yyyymm'] = signals['fdate'].dt.strftime('%Y%m').astype(int)


In [116]:
merged_df = pd.merge(
    merged_df, 
    signals, 
    on=['yyyymm', 'permno'], 
    how='inner',
    suffixes=('', '_signals')
)

In [117]:
missing_percentage = merged_df.groupby('permno').apply(
    lambda group: group.isnull().mean() * 100
)
# filter out companies that dont have 0% missing data for ticker in merged_df, meaning they might have been delisted or are newly listed
valid_permnos = missing_percentage[missing_percentage['ticker'] == 0].index
merged_df = merged_df[merged_df['permno'].isin(valid_permnos)]

# for each permno, check if it has data for yyyymm starting from 199501
valid_permnos = merged_df.groupby('permno')['yyyymm'].min().reset_index()
valid_permnos = valid_permnos[valid_permnos['yyyymm'] <= 199501]['permno']
merged_df = merged_df[merged_df['permno'].isin(valid_permnos)]

# for each permno, check if it has data for yyyymm that ends in 201912
valid_permnos = merged_df.groupby('permno')['yyyymm'].max().reset_index()
valid_permnos = valid_permnos[valid_permnos['yyyymm'] >= 201912]['permno']
merged_df = merged_df[merged_df['permno'].isin(valid_permnos)]
print(valid_permnos.shape[0])

272


/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/1942721390.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_percentage = merged_df.groupby('permno').apply(


## Mean Reversion Factor

In [118]:
# Create a mean reversion signal by calculating rolling z-scores of returns
def calculate_z_score(series):
    # Convert numpy array to pandas Series if needed
    if isinstance(series, np.ndarray):
        series = pd.Series(series)
    
    if len(series) == 0 or series.isna().all():
        return np.nan
    mean = series.mean()
    std = series.std()
    # Handle division by zero
    if std == 0:
        return np.nan
    # Return the z-score of the last value in the series
    return (series.iloc[-1] - mean) / std

# Sort dataframe by permno and date for proper time series analysis
merged_df = merged_df.sort_values(['permno', 'yyyymm'])

# Calculate returns by permno
merged_df['ret'] = merged_df.groupby('permno')['PRC'].pct_change()

# Create rolling z-scores by permno
z_scores = []
for permno, group in merged_df.groupby('permno'):
    group = group.sort_values('yyyymm')
    group['rolling_z_score'] = group['ret'].rolling(window=36).apply(calculate_z_score, raw=False)
    z_scores.append(group)

# Combine results
merged_df = pd.concat(z_scores)

# Clean up z-scores
merged_df['rolling_z_score'] = merged_df['rolling_z_score'].fillna(0)
merged_df['rolling_z_score'] = merged_df['rolling_z_score'].replace([np.inf, -np.inf], 0)

# Create mean reversion signal
merged_df['mean_reversion_signal'] = np.where(
    merged_df['rolling_z_score'] > 1, -1,
    np.where(merged_df['rolling_z_score'] < -1, 1, 0)
)

## Macro Uncertainty

In [119]:
# Macro Uncertainty
macro_uncertainty_original = pd.read_sas("macro.sas7bdat", encoding='ISO-8859-1')

In [120]:
macro_uncertainty = macro_uncertainty_original.copy()
macro_uncertainty["yyyymm"] = macro_uncertainty["date"].dt.strftime('%Y%m').astype(int)
macro_uncertainty.set_index("yyyymm", inplace=True)

merged_df = pd.merge(
    merged_df,
    macro_uncertainty,
    left_on="yyyymm",
    right_index=True,
    how="left",
) 

In [121]:
# Winsorization using groupby and vectorized operations
non_data_cols = {'permno', 'yyyymm', 'monthid', 'ticker', 'conm', 'gvkey', 'cusip', 'naics', 'gsubind', 'PRC', 'VOL', 'RET', 'SHROUT'}
data_cols = set(merged_df.columns) - non_data_cols
# Winsorization using groupby and avoiding fragmentation
def winsorize(group):
    group = group.copy()  # Avoid modifying the original group
    winsorized_data = {}  # Collect winsorized columns here
    for column in data_cols:
        lower_quantile = group[column].quantile(0.01)
        upper_quantile = group[column].quantile(0.99)
        winsorized_data[f'{column}_winsorized'] = group[column].clip(lower=lower_quantile, upper=upper_quantile)
    # Combine the original group with the new winsorized columns
    winsorized_df = pd.concat([group, pd.DataFrame(winsorized_data, index=group.index)], axis=1)
    return winsorized_df

# Apply Winsorization by grouping on 'monthid'
merged_df = merged_df.groupby('monthid', group_keys=False).apply(winsorize)

print(merged_df.head())

    permno    yyyymm  monthid ticker         conm   gvkey      cusip   naics  \
0  10104.0  199501.0    181.0   ORCL  ORACLE CORP  012142  68389X105  519130   
1  10104.0  199502.0    182.0   ORCL  ORACLE CORP  012142  68389X105  519130   
2  10104.0  199503.0    183.0   ORCL  ORACLE CORP  012142  68389X105  519130   
3  10104.0  199504.0    184.0   ORCL  ORACLE CORP  012142  68389X105  519130   
4  10104.0  199505.0    185.0   ORCL  ORACLE CORP  012142  68389X105  519130   

    gsubind        IM  ...  divinc_prob_winsorized  \
0  45103020 -0.015804  ...                     NaN   
1  45103020 -0.034445  ...                     NaN   
2  45103020 -0.007065  ...                     NaN   
3  45103020  0.005445  ...                     NaN   
4  45103020  0.031876  ...                     NaN   

   rolling_z_score_winsorized  KDJ_120_winsorized  h_1_winsorized  \
0                         0.0            0.489046        0.572315   
1                         0.0            0.479945       

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/913607033.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('monthid', group_keys=False).apply(winsorize)


In [122]:
def categorize_factors(df, factor_columns, skew_threshold=0.5, kurt_threshold=3.0):
    """
    Automatically categorizes factors based on their statistical properties.
    
    Args:
        df: DataFrame with your factor data
        factor_columns: List of column names for factors
        skew_threshold: Absolute skewness threshold to consider a distribution skewed
        kurt_threshold: Kurtosis threshold beyond normal (normal = 3)
    
    Returns:
        Dictionary with 'zscore_factors' and 'percentile_factors' lists
    """
    zscore_factors = []
    percentile_factors = []
    
    for factor in factor_columns:
        grouped = df.groupby('yyyymm')[factor]
        avg_skew = grouped.apply(lambda x: skew(x.dropna())).mean()
        avg_kurt = grouped.apply(lambda x: kurtosis(x.dropna(), fisher=False)).mean()
        
        # Decide based on the statistics
        if (abs(avg_skew) > skew_threshold or 
            avg_kurt > kurt_threshold + 3):
            percentile_factors.append(factor)
        else:
            zscore_factors.append(factor)
    
    return {
        'zscore_factors': zscore_factors,
        'percentile_factors': percentile_factors
    }

In [ ]:
factor_columns = merged_df.select_dtypes(include=[np.number]).columns[208:] # <-- PROBLEM: THIS IS NOT ALL THE COLUMNS WE WANT
factor_categories = categorize_factors(merged_df, factor_columns)
factor_categories

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/2431557147.py:19: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  avg_skew = grouped.apply(lambda x: skew(x.dropna())).mean()
/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/2431557147.py:20: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  avg_kurt = grouped.apply(lambda x: kurtosis(x.dropna(), fisher=False)).mean()
/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/2431557147.py:19: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  avg_skew = grouped.apply(lambda x: skew(x.dropna())).mean()
/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/2431557147.py:20: Runti

{'zscore_factors': ['CAPES_winsorized',
  'deviation_pct120_winsorized',
  'DBREADTH_winsorized',
  'roa_winsorized',
  'ret_f1_winsorized',
  'xret_10_winsorized',
  'dBlock_N_winsorized',
  'ret_f11_winsorized',
  'mean_reversion_signal_winsorized',
  'sue_winsorized',
  'CF_winsorized',
  'str_mod_winsorized',
  'FIRMTANG_winsorized',
  'ret_f7_winsorized',
  'ret_f12_winsorized',
  'xret_5_winsorized',
  'MoneyFlowIndex_20_winsorized',
  'lag_log_size_winsorized',
  'RSI_120_winsorized',
  'seasonality_winsorized',
  'ret_f8_winsorized',
  'ret_f3_winsorized',
  'ret_f6_winsorized',
  'ret_f2_winsorized',
  'h_3_winsorized',
  'FCF_winsorized',
  'O_score_winsorized',
  'ret_f5_winsorized',
  'xret_indsize_120_winsorized',
  'xret_120_winsorized',
  'Accrual_winsorized',
  'momentum_winsorized',
  'deviation_pct20_winsorized',
  'trend_factor_winsorized',
  'ret_f9_winsorized',
  'leverage_winsorized',
  'ret_f4_winsorized',
  'IM_winsorized',
  'RSI_20_winsorized',
  'sue_NI_winso

In [124]:
for factor in factor_categories['zscore_factors']:
    merged_df[factor] = merged_df.groupby(['yyyymm'])[factor].transform(
        lambda x: (x - x.mean()) / x.std() if x.std() != 0 else 0
    )

for factor in factor_categories['percentile_factors']:
    merged_df[f'{factor}_std'] = merged_df.groupby(['yyyymm'])[factor].transform(
        lambda x: x.rank(pct=True)
    )

merged_df

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/3804302227.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[f'{factor}_std'] = merged_df.groupby(['yyyymm'])[factor].transform(
/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/3804302227.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[f'{factor}_std'] = merged_df.groupby(['yyyymm'])[factor].transform(
/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/3804302227.py:7: PerformanceWarning: DataFrame is highly 

,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,IV_capm_winsorized_std,DIV_P_winsorized_std,QUICK_winsorized_std,RECTURN_winsorized_std,divinc_prob_winsorized_std,specdiv_prob_winsorized_std,mdr_winsorized_std,PMSG_winsorized_std,CFM_winsorized_std,EV_EBITDA_winsorized_std
0,10104.0,199501.0,181.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.015804,...,0.926471,0.080882,0.267925,0.200000,NaN,NaN,0.963235,0.159533,0.819853,0.083333
1,10104.0,199502.0,182.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.034445,...,0.822878,0.079044,0.251880,0.203774,NaN,NaN,0.686347,0.176030,0.794118,0.063725
2,10104.0,199503.0,183.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.007065,...,0.630996,0.079044,0.251880,0.203774,NaN,NaN,0.693727,0.172285,0.794118,0.059113
3,10104.0,199504.0,184.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,0.005445,...,0.761029,0.079044,0.251880,0.200000,NaN,NaN,0.755515,0.198502,0.779412,0.058824
4,10104.0,199505.0,185.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,0.031876,...,0.929889,0.079044,0.263158,0.192453,NaN,NaN,0.955720,0.209738,0.790441,0.056391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147683,91556.0,201908.0,476.0,ROST,ROSS STORES INC,009248,778296103,448140,25504010,0.082175,...,0.242647,0.180147,0.758364,0.996296,0.839130,NaN,0.639706,0.370370,0.426471,0.183824
147684,91556.0,201909.0,477.0,ROST,ROSS STORES INC,009248,778296103,448140,25504010,0.076389,...,0.419118,0.169118,0.758364,0.996296,0.830435,NaN,0.566176,0.400000,0.426471,0.125000
147685,91556.0,201910.0,478.0,ROST,ROSS STORES INC,009248,778296103,448140,25504010,0.034279,...,0.448529,0.180147,0.758364,0.996296,0.826087,NaN,0.569853,0.392593,0.426471,0.128676
147686,91556.0,201911.0,479.0,ROST,ROSS STORES INC,009248,778296103,448140,25504010,-0.003040,...,0.441176,0.172794,0.773234,0.996296,0.813043,NaN,0.198529,0.354244,0.422794,0.148148


In [125]:
def calculate_univariate_power(df, factor_cols):
    results = {
        'factor': [],
        'avg_coef': [],
        't_stat': [],
        'p_value': []
    }
    months = sorted(df['yyyymm'].unique())
    
    # For each factor, run monthly cross-sectional regressions
    for factor in factor_cols:
        coefs = []
        
        for month in months:
            month_data = df[df['yyyymm'] == month].copy()
            
            if len(month_data) < 10: # At least 10 data points before proceeding
                   continue
            
            X = month_data[[factor]].copy()
            X = sm.add_constant(X)
            y = month_data['ret_f1']
            if X.isnull().values.any() or y.isnull().values.any():
                   continue
            
            model = sm.OLS(y, X).fit()
            if len(model.params) < 2:
                   continue
            coefs.append(model.params[1])  # Index 1 is the factor coefficient (0 is intercept)
        
        if len(coefs) > 0:
            avg_coef = np.mean(coefs)
            t_stat = (avg_coef / np.std(coefs)) * np.sqrt(len(coefs))
            p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=len(coefs)-1))
            
            results['factor'].append(factor)
            results['avg_coef'].append(avg_coef)
            results['t_stat'].append(t_stat)
            results['p_value'].append(p_value)
    
    results_df = pd.DataFrame(results)
    results_df['abs_t_stat'] = results_df['t_stat'].abs()
    results_df = results_df.sort_values('abs_t_stat', ascending=False)
    
    return results_df

In [126]:
factor_power = calculate_univariate_power(merged_df, factor_columns)
factor_power.head(30)

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/3146116784.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefs.append(model.params[1])  # Index 1 is the factor coefficient (0 is intercept)
/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/3146116784.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefs.append(model.params[1])  # Index 1 is the factor coefficient (0 is intercept)
/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_32227/3146116784.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will 

KeyboardInterrupt: 

In [ ]:
top_factors = factor_power.head(30) # <-- TEMPORARY

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot t-statistics of top 30 factors
plt.figure(figsize=(12, 8))
top_factors = factor_power.head(30)
plt.barh(top_factors['factor'], top_factors['t_stat'])
plt.axvline(x=1.96, color='r', linestyle='--', label='95% significance')
plt.axvline(x=-1.96, color='r', linestyle='--')
plt.title('Predictive Power of Top 30 Factors')
plt.xlabel('t-statistic')
plt.ylabel('Factor')
plt.grid(axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# Doesn't work yet...

def analyze_factor_correlations(df, top_factors):
    factor_data = df[top_factors]
    
    corr_matrix = factor_data.corr()
    
    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, 
                linewidths=0.5, fmt=".2f")
    plt.title('Correlation Matrix of Top Factors')
    plt.tight_layout()
    plt.show()
    
    return corr_matrix

def identify_correlated_pairs(corr_matrix, threshold=0.7):
    correlated_pairs = []
    
    # Upper triangle of the correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    for col in upper.columns:
        for idx in upper.index:
            if abs(upper.loc[idx, col]) > threshold:
                correlated_pairs.append({
                    'factor1': idx,
                    'factor2': col,
                    'correlation': upper.loc[idx, col]
                })
    
    return pd.DataFrame(correlated_pairs).sort_values('correlation', key=abs, ascending=False)